In [1]:
import time
import cv2
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from numpy import random
import numpy as np
import math
import torchvision
import re

In [2]:
!pip install easyocr --quiet
import easyocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.2/619.2 KB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 KB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 16.1 MB/s eta 0:00:00


In [3]:
!git clone https://github.com/WongKinYiu/yolov7.git
%cd yolov7

Cloning into 'yolov7'...
remote: Enumerating objects: 1127, done.
remote: Total 1127 (delta 0), reused 0 (delta 0), pack-reused 1127
Receiving objects: 100% (1127/1127), 69.93 MiB | 24.77 MiB/s, done.
Resolving deltas: 100% (522/522), done.
/content/yolov7


In [4]:
from models.experimental import attempt_load
from utils.general import check_img_size, non_max_suppression, \
    scale_coords, xyxy2xywh, strip_optimizer, set_logging, increment_path
from utils.torch_utils import time_synchronized
from utils.datasets import letterbox

In [5]:
def detect(model, source):

    device = 'cpu'
    imgsz = 640
    img = source


    stride = int(model.stride.max())  # model stride
    imgsz = check_img_size(imgsz, s=stride)  # check img_size TODO

    names = model.module.names if hasattr(model, 'module') else model.names
    old_img_w = old_img_h = imgsz
    old_img_b = 1
    im0 = img

        # Convert
    img = letterbox(img, imgsz, stride=stride)[0]
    img = img[:, :, ::-1].transpose(2, 0, 1)  # BGR to RGB, to 3x416x416
    img = np.ascontiguousarray(img)


    img = torch.from_numpy(img).to(device)
    img = img.float()  # uint8 to fp16/32
    img /= 255.0  # 0 - 255 to 0.0 - 1.0
    if img.ndimension() == 3:
        img = img.unsqueeze(0)

    with torch.no_grad():   # Calculating gradients would cause a GPU memory leak
        pred = model(img)[0]
    # Apply NMS
    pred = non_max_suppression(pred, 0.25, 0.45, classes=[0,1,2,3,4,5])

    # Process detections
    outputs = []
    for i, det in enumerate(pred):  # detections per image

        if len(det):

            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

            for *xyxy, conf, cls in reversed(det):
                label = f'{names[int(cls)]}'
                out = [t.item() for t in xyxy]
                outputs.append((out, label, conf.item()))

    return outputs

def crop_image(out, img):
  x1 = int(out[0][0])
  x2 = int(out[0][2])
  y1 = int(out[0][1])
  y2 = int(out[0][3])
  # Cropping an image
  cropped_image = img[y1:y2, x1:x2]

  return cropped_image

In [6]:
!pip install --upgrade --no-cache-dir gdown --quiet
!gdown 1qXBtVeOpZjeY-AduqtyTtu22y6RytGeO
!gdown 1Ni_w3QCucfHI271ASUjk7k04GZseGJcz

Downloading...
From: https://drive.google.com/uc?id=1qXBtVeOpZjeY-AduqtyTtu22y6RytGeO
To: /content/yolov7/classify.pt
100% 74.8M/74.8M [00:00<00:00, 99.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Ni_w3QCucfHI271ASUjk7k04GZseGJcz
To: /content/yolov7/segmentation.pt
100% 74.8M/74.8M [00:01<00:00, 56.5MB/s]


In [7]:
model1 = attempt_load('segmentation.pt')
model2 = attempt_load('classify.pt')
reader = easyocr.Reader(['en'])

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
Fusing layers... 


RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [8]:
def inference(image_path:str):
  '''
  Function responsible for inference.
  Args: 
    image_path: str, path to image file. eg. "input/aveksha_micu_mon--209_2023_1_17_12_0_34.jpeg"
  Returns:
    result: dict, final output dictionary. eg. {"HR":"80", "SPO2":"98", "RR":"15", "SBP":"126", "DBP":"86"}
  '''
  result = {}
  img = cv2.imread(image_path)
  
  out1 = detect(model1, img)
  out1_img = crop_image(out1[0], img)

  out2 = detect(model2, out1_img)
  temp = {}
  
  for out in out2:
    out2_img = crop_image(out, out1_img)

    r = reader.readtext(out2_img)
    if(not r):
      continue
    r = r[0]

    if(r[2] < 0.2):
      continue

    tt = temp.get(out[1])
    if(tt):
      if tt['class_conf'] > out[2]:
        continue


    temp[out[1]] = {
          'class_conf': out[2],
          'value': re.sub("[^0-9]", "", r[1]),
          'value_conf': r[2]
      }

  l = ['HR', 'SPO2', 'RR', 'SBP', 'DBP', 'MAP']

  for i in l:
    if(not temp.get(i)):
      continue

    result[i] = temp[i]['value']

  return result